## 模型搭建

In [2]:
from cobra import Model, Reaction, Metabolite

### 创建模型、反应、代谢物

In [3]:
model = Model('test_model')
reaction = Reaction('R_3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [4]:
ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    'M3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')

### 添加反应和GPR

In [12]:
reaction.add_metabolites({
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})

reaction  # This gives a string representation of the reaction

Reaction identifier,R_3OAS140
Name,3 oxoacyl acyl carrier protein synthase n C140
Memory address,0x7f142b296350
Stoichiometry,ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c Dodecanoyl-ACP-n-C120ACP + H + Malonyl-acyl-carrier-protein --> acyl-carrier-protein + 3-Oxotetradecanoyl-acyl-carrier-protein + CO2
GPR,
Lower bound,0.0
Upper bound,1000.0


In [18]:
reaction.gene_reaction_rule = '( STM2378 or STM1197 )'
reaction

Reaction identifier,R_3OAS140
Name,3 oxoacyl acyl carrier protein synthase n C140
Memory address,0x7f142b296350
Stoichiometry,ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c Dodecanoyl-ACP-n-C120ACP + H + Malonyl-acyl-carrier-protein --> acyl-carrier-protein + 3-Oxotetradecanoyl-acyl-carrier-protein + CO2
GPR,STM2378 or STM1197
Lower bound,0.0
Upper bound,1000.0


In [21]:
model.add_reactions([reaction])

# The objects have been added to the model
print(f'{len(model.reactions)} reactions')
print(f'{len(model.metabolites)} metabolites')
print(f'{len(model.genes)} genes')

1 reactions
6 metabolites
2 genes


In [48]:
model.objective = 'R_3OAS140'
print(model.objective_direction)
model.objective.expression

max


1.0*R_3OAS140 - 1.0*R_3OAS140_reverse_60acb

### 模型有效性

In [73]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)

# 保存SBML
# write_sbml_model(model, "test_model.xml")

(<Model test_model at 0x7f1429666e30>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': [],
  'SBML_FATAL': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': []})


### 添加虚拟反应

In [58]:
model.add_metabolites([
    Metabolite(
    'glycogen_c',
    name='glycogen',
    compartment='c'
    ),
    Metabolite(
    'co2_e',
    name='CO2',
    compartment='e'
    ),
])

In [67]:
# create exchange reaction 胞外
model.add_boundary(model.metabolites.get_by_id("co2_e"), type="exchange")

Reaction identifier,EX_co2_e
Name,CO2 exchange
Memory address,0x7f1429666530
Stoichiometry,co2_e <=> CO2 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [68]:
# create exchange reaction 胞内
model.add_boundary(model.metabolites.get_by_id("glycogen_c"), type="sink")

Reaction identifier,SK_glycogen_c
Name,glycogen sink
Memory address,0x7f1429665390
Stoichiometry,glycogen_c <=> glycogen <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [71]:
model.reactions

[<Reaction R_3OAS140 at 0x7f142b296350>,
 <Reaction EX_co2_e at 0x7f1429666530>,
 <Reaction SK_glycogen_c at 0x7f1429665390>]

In [ ]:
# 保存模型用于Escher
# load_json_model('PATH')
save_json_model(model, "test.json")

## 模型使用

In [2]:
from cobra.io import load_model
import cobra
model = load_model("textbook")

In [3]:
solution = model.optimize()
print(solution)

<Solution 0.874 at 0x747707e5b040>


In [4]:
%%time
# 全FBA
model.optimize().objective_value

CPU times: user 876 μs, sys: 57 μs, total: 933 μs
Wall time: 940 μs


0.8739215069684305

In [5]:
%%time
# 单FBA
model.slim_optimize()

CPU times: user 157 μs, sys: 10 μs, total: 167 μs
Wall time: 172 μs


0.8739215069684305

In [6]:
# 模拟报告
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


In [7]:
# 单代谢物汇报
model.metabolites.nadh_c.summary()

Percent,Flux,Reaction,Definition
13.14%,5.064,AKGDH,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
8.04%,3.1,Biomass_Ecoli_core,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
41.58%,16.02,GAPD,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
13.14%,5.064,MDH,mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
24.09%,9.283,PDH,coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
Percent,Flux,Reaction,Definition
100.00%,-38.53,NADH16,4.0 h_c + nadh_c + q8_c --> 3.0 h_e + nad_c + q8h2_c


In [8]:
model.reactions.AKGDH.summary()

In [9]:
# 细胞基本组分构成目标函数
model.reactions.Biomass_Ecoli_core.reaction

'1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c'

In [10]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

{<Reaction Biomass_Ecoli_core at 0x747708119c00>: 1.0}

In [11]:
model.reactions.ATPM

# change the objective to ATPM
model.objective = "ATPM"

# The upper bound should be 1000, so that we get
# the actual optimal value
model.reactions.get_by_id("ATPM").upper_bound = 1000.
linear_reaction_coefficients(model)

{<Reaction ATPM at 0x74770811a680>: 1.0}

In [12]:
%%time
model.slim_optimize()

CPU times: user 539 μs, sys: 31 μs, total: 570 μs
Wall time: 576 μs


175.00000000000006

In [13]:
# FVA分析
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(model, model.reactions[:5], fraction_of_optimum=0.9)

,minimum,maximum
ACALD,-2.692308,0.0
ACALDt,-2.692308,0.0
ACKr,-4.117647,0.0
ACONTa,8.461538,20.0
ACONTb,8.461538,20.0


In [36]:
# 通用配置FVA
loop_reactions = [model.reactions.FRD7, model.reactions.SUCDi]
flux_variability_analysis(model, model.reactions.FRD7, fraction_of_optimum=0.9, loopless=True)

,minimum,maximum
FRD7,0.0,0.0


In [48]:
model.summary(fva=0.9)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,[9; 10],6,100.00%
o2_e,EX_o2_e,60,[51.76; 60],0,0.00%
Metabolite,Reaction,Flux,Range,C-Number,C-Flux
ac_e,EX_ac_e,0,[-4.118; 0],2,0.00%
acald_e,EX_acald_e,0,[-2.692; 0],2,0.00%
akg_e,EX_akg_e,0,[-1.489; 0],5,0.00%
co2_e,EX_co2_e,-60,[-60; -48.33],1,100.00%
etoh_e,EX_etoh_e,0,[-2.333; 0],2,0.00%
for_e,EX_for_e,0,[-11.67; 0],1,0.00%
glu__L_e,EX_glu__L_e,0,[-1.346; 0],5,0.00%


In [ ]:
model.objective = 'Biomass_Ecoli_core'
fba_solution = model.optimize()
pfba_solution = cobra.flux_analysis.pfba(model)

In [66]:
pfba_solution.fluxes

ACALD     0.000000e+00
ACALDt    0.000000e+00
ACKr     -2.484585e-14
ACONTa    6.007250e+00
ACONTb    6.007250e+00
              ...     
TALA      1.496984e+00
THD2      0.000000e+00
TKT1      1.496984e+00
TKT2      1.181498e+00
TPI       7.477382e+00
Name: fluxes, Length: 95, dtype: float64

In [70]:
geometric_fba_sol = cobra.flux_analysis.geometric_fba(model)
geometric_fba_sol

,fluxes,reduced_costs
ACALD,0.000000e+00,0.0
ACALDt,0.000000e+00,0.0
ACKr,1.260427e-13,0.0
ACONTa,6.007250e+00,0.0
ACONTb,6.007250e+00,0.0
...,...,...
TALA,1.496984e+00,0.0
THD2,0.000000e+00,0.0
TKT1,1.496984e+00,0.0
TKT2,1.181498e+00,0.0
